<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/04_02_linear_models_SLIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка


**Цель:**
- Реализовать, настроить и оценить модель SLIM (Sparse Linear Methods for Top-N Recommendations).


**Данные:**
- используем train(разбиение на v_train, val), test  из предыдущих ноутбуков.
- вход - бинаризованная пользователь-айтем матрица (implicit feedback)


## 01. Клонируем репозиторий. Подключаем GoogleDrive.

In [1]:
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project

Cloning into 'Recommender_Systems_project'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 133 (delta 66), reused 51 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 643.09 KiB | 4.98 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
# подключаем диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.4 MB/s eta 0:00:00


## 02. Импорты

In [13]:
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import optuna
from optuna.samplers import TPESampler

import time
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint
import numbers

from joblib import Parallel, delayed

from scipy.sparse import csr_matrix, load_npz
from sklearn.linear_model import ElasticNet

# Внутренние модули
from recommender.data_io import train_test_reader                 # для чтения сохранённых из 001_data_and_eda_1m_proba
from recommender.splitters import df_time_split                   # для time_split разбиения df
from recommender.preprocessing import prepare_ui_matrix           # для получения матрицы взаимодействий
from recommender.metrics import hitrate_at_k, coverage_at_k, precision_at_k,\
                     recall_at_k, ndcg_at_k, map_at_k, model_evaluation
from recommender.results_logger import save_experiment_results    # для сохранения результатов

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA = Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA/"raw"
PROCESSED = DATA / "processed"
RESULTS_DIR = DATA / "results"
DATA.mkdir(exist_ok=True)

TOP_N = 10

## 03. Грузим train, test, meta_данные.



In [5]:
train_test_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'

train, test, meta = train_test_reader(train_test_path)
pprint(meta, width=80, compact=False)
print(f'\ntrain shape : {train.shape}')
print(f'test shape  : {test.shape}')
print( '\n', '*'*50, '\ntrain.head')
display(train.head(3))
print('\n', '*'*50, '\ntest.head')
display(test.head(3))

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

train shape : (800142, 6)
test shape  : (94842, 6)

 ************************************************** 
train.head


,user_id,item_id,rating,timestamp,title,genres
0,635,1251,4,975768620,8 1/2 (1963),Drama
1,635,3948,4,975768294,Meet the Parents (2000),Comedy
2,635,1270,4,975768106,Back to the Future (1985),Comedy|Sci-Fi



 ************************************************** 
test.head


,user_id,item_id,rating,timestamp,title,genres
0,635,3789,5,975768788,"Pawnbroker, The (1965)",Drama
1,635,2987,5,979141847,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
2,635,2988,4,975769007,Melvin and Howard (1980),Drama


## 04. Бьем Train на v_train и val для подбора гиперпараметров
Используем опять вариант **B** - разбиение по времени(Time-based split).
 Все события до порога в v_train, события после в val. Это гарантирует, что ни одно событие из "будущего" не попадёт в v_train.

In [6]:
# Бьем Train на v_train и val
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
user_id = 'user_id'
item_id = 'item_id'
time_column = 'timestamp'

# минимальное тёплое число оценок
min_n_reitings = 5
# min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
quantile = 0.85    # квантиль разбиения
# разбиение
v_train, val = df_time_split(train, time_column, columns_to_save,
                             user_id , item_id,
                             min_n_reitings, n, k , quantile)

users_intersection = set(val['user_id']) & set(v_train['user_id'])
print('*'*70)
print(f"\n{len(users_intersection)} юзера встречаются одновременно и в v_train и в val")

# залистим тестовых пользователей
val_users = sorted(users_intersection)

print('\ntrain ')
display(v_train.head(2))
print('\ntest ')
display(val.head(2))

Порог разбиения по времени 974814617
Размеры после разбиения: train 680122 test 120020

Осталось пользователей в train : 4748
Новый размер train: (680108, 6)
Осталось пользователей в test: 211
Новый размер test: (26076, 6)
**********************************************************************

211 юзера встречаются одновременно и в v_train и в val

train 


,user_id,item_id,rating,timestamp,title,genres
0,1272,720,5,974814383,Wallace & Gromit: The Best of Aardman Animatio...,Animation
1,1272,2993,2,974814617,Thunderball (1965),Action



test 


,user_id,item_id,rating,timestamp,title,genres
0,1272,2987,4,974822814,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
1,1272,2989,4,974814713,For Your Eyes Only (1981),Action


## 05.Подготовка  матрицы и словарей
Используем то же , что и в KNN/ALS.

In [7]:
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
v_train_matrix = load_npz(input_dir / "v_train_matrix.npz")
# train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index_v.pkl", "rb") as f:
    user2index_v = pickle.load(f)

with open(input_dir / "item2index_v.pkl", "rb") as f:
    item2index_v = pickle.load(f)

with open(input_dir / "index2user_v.pkl", "rb") as f:
    index2user_v = pickle.load(f)

with open(input_dir / "index2item_v.pkl", "rb") as f:
    index2item_v = pickle.load(f)

assert isinstance(v_train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"

In [11]:
# заменяем реальные ID на индексы из матрицы обучения модели
val_mapped = val.assign(
    user_id = val["user_id"].map(user2index_v),
    item_id = val["item_id"].map(item2index_v)
)
assert val_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
val_items = val_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items для coverage
all_items = set(v_train['item_id'].map(item2index_v).dropna().astype(int).unique())

# список пользователей для валидации
val_users_index = list(val_items.keys())

## 06. Реализация SLIM
- реализация SLIM с регуляризацией ElasticNet и топ-K разрежением матрицы сходства между айтемами.

In [21]:
class PET_SLIM:
    '''

    '''
    def __init__(self, alpha=1.0, l1_ratio=0.01, topK=200, max_iter = 200, n_jobs=-1):
        self.alpha = alpha          # сила регуляризации
        self.l1_ratio = l1_ratio    # доля L1 в ElasticNet (0 -  L2, 1 - L1)
        self.topK = topK            # число ненулевых соседей
        self.n_jobs = n_jobs        # число потоков
        self.W = None               # матрица весов item x item
        self.train_matrix = None    # CSR матрица взаимодействий users x items
        self.max_iter = max_iter    # максимум итераций оптимизатора ElasticNet

    def _fit_item(self, j, X, n_items):
        y = X[:, j].toarray().ravel()

        # исключаем сам столбец j из признаков
        mask = np.ones(n_items, dtype=bool)
        mask[j] = False
        X_j = X[:, mask]

        # Модель ElasticNet с положительными коэффициентами и без bias
        model = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio,
                           positive=True, fit_intercept=False,
                           max_iter=self.max_iter, random_state=42)
        model.fit(X_j, y)

        # Собираем полный вектор коэффициентов длины n_items,
        # заполняя места признаков оцененными весами, место j останется 0
        coef = np.zeros(n_items)
        coef[mask] = model.coef_

        # оставляем только K крупнейших коэффициентов
        if self.topK is not None and self.topK < n_items:
            idx = np.argpartition(-coef, self.topK)[:self.topK]
            mask_top = np.zeros_like(coef, dtype=bool)
            mask_top[idx] = True
            coef[~mask_top] = 0.0
        # обнуляем самосвязь
        coef[j] = 0.0
        return coef

    def fit(self, train_matrix):
        self.train_matrix = train_matrix.tocsr()
        n_items = self.train_matrix.shape[1]


        # параллельно обучаем каждый столбец
        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit_item)(j, self.train_matrix, n_items)
            for j in range(n_items)
        )
        # формируем матрицу W
        self.W = np.vstack(results).T   # item × item

    def recommend_one(self, user_id, N=10):
        # вектор взаимодействий пользователя
        u = self.train_matrix[user_id].toarray().ravel()

        scores = u @ self.W

        # маскируем уже взаимодействованные айтемы
        scores[u > 0] = -np.inf

        # топ-N индексов
        top_idx = np.argpartition(-scores, N)[:N]
        top_scores = scores[top_idx]

        # сортируем
        order = np.argsort(-top_scores)
        return top_idx[order], top_scores[order]

    def recommend_batch_matrix(self, user_ids, N=10):
        # матрица взаимодействий для набора пользователей
        U = self.train_matrix[user_ids].toarray()

        # батч-скоринг
        scores = U @ self.W

        # маскировка уже виденных айтемов поэлементно
        scores[U > 0] = -np.inf

        # топ-N для каждого пользователя
        top_items = np.argpartition(-scores, N, axis=1)[:, :N]
        top_scores = np.take_along_axis(scores, top_items, axis=1)

        # сортировка
        order = np.argsort(-top_scores, axis=1)
        items_sorted = np.take_along_axis(top_items, order, axis=1)
        scores_sorted = np.take_along_axis(top_scores, order, axis=1)
        return items_sorted, scores_sorted

    def recommend(self, users_id, N=10):
        if isinstance(users_id, numbers.Number):
            return self.recommend_one(users_id, N)
        elif isinstance(users_id, list) or isinstance(users_id, np.ndarray):
            return self.recommend_batch_matrix(users_id, N)
        else:
            raise ValueError("users_id должен быть int или list[int]")


## 07. подбор гиперпараметров с Optuna


In [22]:
# подбор гиперпараметров Optuna
def make_objective_slim(val_items, X_ui, metric_fn=ndcg_at_k, k_eval=TOP_N):
    val_users_index = list(val_items.keys())
    def objective(trial):
        alpha = trial.suggest_float("alpha", 1e-4, 1e2, log=True)
        l1_ratio = trial.suggest_float("l1_ratio", 1e-4, 1.0)
        topK = trial.suggest_int("topK", 200, 1000)
        max_iter = trial.suggest_int("max_iter", 200, 1000)

        model = PET_SLIM(alpha=alpha, l1_ratio=l1_ratio, topK=topK, max_iter=max_iter)
        model.fit(X_ui)

        items, ___it_scores____ = model.recommend(val_users_index,  N=k_eval)
        items = items.tolist()
        recs = {val_users_index[i]: items[i] for i in range(len(val_users_index))}

        score = metric_fn(recs, val_items, k=k_eval)
        return score
    return objective


In [23]:
n_trials = 30
# замерим время
start = time.time()

sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

study_slim = optuna.create_study(direction="maximize", sampler=sampler)
study_slim.optimize(make_objective_slim(val_items, v_train_matrix), n_trials=n_trials)

end = time.time()
study_slim_time = end - start

print('#'*70)
print("\nЛучшие параметры SLIM:", study_slim.best_params)
print("Лучший NDCG@10:", study_slim.best_value)

print('\n')
print('*'*70)
print(f"Время выполнения: {study_slim_time:.2f} секунд")

[I 2025-12-14 15:29:36,601] A new study created in memory with name: no-name-c27eb16a-36e1-4038-8d52-02db751eedc8
[I 2025-12-14 15:31:21,533] Trial 0 finished with value: 0.27592284181862914 and parameters: {'alpha': 0.017670169402947963, 'l1_ratio': 0.9507192349792751, 'topK': 786, 'max_iter': 679}. Best is trial 0 with value: 0.27592284181862914.
[I 2025-12-14 15:34:25,927] Trial 1 finished with value: 0.2580198084200377 and parameters: {'alpha': 0.0008632008168602544, 'l1_ratio': 0.15607892088416903, 'topK': 246, 'max_iter': 893}. Best is trial 0 with value: 0.27592284181862914.
[I 2025-12-14 15:35:25,853] Trial 2 finished with value: 0.08124313856723395 and parameters: {'alpha': 0.4042872735027334, 'l1_ratio': 0.7081017705382658, 'topK': 216, 'max_iter': 976}. Best is trial 0 with value: 0.27592284181862914.
[I 2025-12-14 15:36:25,410] Trial 3 finished with value: 0.020286363535450158 and parameters: {'alpha': 9.877700294007917, 'l1_ratio': 0.21241787676720833, 'topK': 345, 'max_it

######################################################################

Лучшие параметры SLIM: {'alpha': 0.4689400963537689, 'l1_ratio': 0.13957991126597663, 'topK': 434, 'max_iter': 493}
Лучший NDCG@10: 0.32393577916657285


**********************************************************************
Время выполнения: 3020.16 секунд


In [24]:
# optuna визуализация
print("OPTUNA for PET_SLIM")
optuna.visualization.plot_optimization_history(study_slim).show()
optuna.visualization.plot_param_importances(study_slim).show()
optuna.visualization.plot_parallel_coordinate(study_slim).show()

OPTUNA for PET_SLIM


###Выводы:
- **SLIM** показал стабильное качество при умеренной регуляризации (alpha ≈ 0.5) и слабой L1-доле (l1_ratio ≈ 0.14). Это говорит о том, что сильная L1‑разреженность ухудшает качество на MovieLens 1M - модель теряет полезные связи.

- Параметр topK = 434 оказался оптимальным:
- - при меньших значениях качество падало (недостаточно соседей),
- - при больших - росло  переобучение.

- Время обучения очень велико...

- По графику важности гиперпараметров видно, что **alpha** - важна, она определяет регуляризацию, l1_ratio, topK, max_iter -вторичны.

## 08. Получение метрик на test

In [25]:
# Получение метрик на test
# загрузка
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index.pkl", "rb") as f:
    user2index = pickle.load(f)

with open(input_dir / "item2index.pkl", "rb") as f:
    item2index = pickle.load(f)

with open(input_dir / "index2user.pkl", "rb") as f:
    index2user = pickle.load(f)

with open(input_dir / "index2item.pkl", "rb") as f:
    index2item = pickle.load(f)

assert isinstance(train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"
train_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 800142 stored elements and shape (5392, 3662)>

In [26]:
# заменяем реальные ID на индексы
test_mapped = test.assign(
    user_id = test["user_id"].map(user2index),
    item_id = test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train['item_id'].map(item2index).dropna().astype(int).unique())

In [27]:
# Вспомним лучшее:
print('Лучшие парамеры SLIM :',study_slim.best_params)

Лучшие парамеры SLIM : {'alpha': 0.4689400963537689, 'l1_ratio': 0.13957991126597663, 'topK': 434, 'max_iter': 493}


In [28]:
best = study_slim.best_params
test_users = list(test_items.keys())
pet_slim = PET_SLIM(best['alpha'], best['l1_ratio'], best['topK'], best['max_iter'])


pet_slim.fit(train_matrix)
# Рекомендации
items, it_scores = pet_slim.recommend(test_users,  N=TOP_N)
items = items.tolist()
slim_recs = {test_users[i]: items[i] for i in range(len(test_users))}

slim_results = model_evaluation(
    slim_recs, test_items, all_items, TOP_N,
    f'slim_alpha_{best['alpha']:.2f}_l1_ratio_{best['l1_ratio']:.2f}'
)
slim_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
slim_alpha_0.47_l1_ratio_0.14,0.825359,0.32201,0.041529,0.342581,0.225772,0.046969


## 09. Сохраняем результаты

In [45]:
# Сохраняем результаты
results_data, json_file, csv_file = save_experiment_results(
    result = slim_results,
    model_name = f'slim_alpha_{best['alpha']:.2f}_l1_ratio_{best['l1_ratio']:.2f}',
    meta=meta,
    results_dir=RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: slim_alpha_0.47_l1_ratio_0.14_20251214_171141.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: slim_alpha_0.47_l1_ratio_0.14
Метка времени: 20251214_171141
Дата оценки: 2025-12-14T17:11:41
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 82.5%
precision@10: 32.20%
recall@10: 4.15%
ndcg@10: 34.26%
map@10: 22.58%
Coverage@10: 4.70%

Последние эксперименты (7 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,"evaluation_date,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date",evaluation_date
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,"2025-11-08T08:49:43.731391,,,,,,,,,",NaN
3,truncated_svd_n_comp=5_n_iter=42,0.851675,0.348206,0.045528,0.365538,0.247310,0.091207,20251116_190936,"2025-11-16T19:09:36.863512,,,,,,,,,",NaN
4,als_factors=5_iter=21_alpha=0.6_reg=0.02,0.840909,0.340191,0.043517,0.358804,0.244510,0.095576,20251210_110349,"2025-12-10T11:03:49.859474,,,,,,,,,",NaN
5,ease_lambda=108727,0.838517,0.338517,0.042712,0.356757,0.243356,0.045603,20251211_185547,"2025-12-11T18:55:47.668555,,,,,,,,,",NaN
6,slim_alpha_0.47_l1_ratio_0.14,0.825359,0.322010,0.041529,0.342581,0.225772,0.046969,20251214_171141,NaN,2025-12-14T17:11:41.060806


# Итого:

## **Выводы:**
- **SLIM** показывает не лучшие результаты при достаточно долгом (строим регрессию для каждого item, а это очень  небысто) обучениии ....